In [22]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from NelsonCheck import NelsonCheck as NC

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import keras

from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D 
from keras import backend as K 

from sklearn import metrics

In [2]:
tagging_df = pd.read_csv("data/cavite_outbreak.csv", parse_dates=['date'])
tagging_df

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4952\3066825937.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  tagging_df = pd.read_csv("data/cavite_outbreak.csv", parse_dates=['date'])


,loc,cases,deaths,date,outbreak
0,CAVITE,64,0,2016-10-01,0
1,CAVITE,214,1,2016-01-17,0
2,CAVITE,161,2,2016-01-24,0
3,CAVITE,147,1,2016-01-31,0
4,CAVITE,160,0,2016-07-02,0
...,...,...,...,...,...
256,CAVITE,60,0,2020-12-13,0
257,CAVITE,60,0,2020-12-20,0
258,CAVITE,57,0,2020-12-27,0
259,CAVITE,68,0,2021-03-01,0


In [3]:
loc_df = tagging_df[tagging_df['loc'] == "CAVITE"]

In [4]:
train = loc_df[loc_df['date'].dt.year < 2019 ]
test = loc_df[loc_df['date'].dt.year == 2019]

In [5]:
train.shape, test.shape, \
"outbreak_train: " + str(train['outbreak'].sum()) + "/" + str(len(train.index)), \
"outbreak_test: " + str(test['outbreak'].sum()) + "/" + str(len(test.index))


((156, 5), (51, 5), 'outbreak_train: 74/156', 'outbreak_test: 29/51')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(train[['cases','deaths','date']], train['outbreak'], test_size=0.33, random_state=42)

In [7]:
x_train_arr = []
x_test_arr = []
city = "CAVITE"
# date = "2018-01-01"
layer = "fapar_daily"
# source = 

for date in X_train.date.values:
    full_date_txt = np.datetime_as_string(date, unit='D')
    data_path = "data/images/" + city +  "/" + layer + "/snapshot-" + full_date_txt + "T00_00_00Z.jpg"
    print(data_path)
    img = cv2.imread(data_path)
    img = cv2.resize(img,(224,224))
    x_train_arr.append(img)

for date in X_test.date.values:
    full_date_txt = np.datetime_as_string(date, unit='D')
    data_path = "data/images/" + city +  "/" + layer + "/snapshot-" + full_date_txt + "T00_00_00Z.jpg"
    img = cv2.imread(data_path)
    img = cv2.resize(img,(224,224))
    x_test_arr.append(img)

data/images/CAVITE/fapar_daily/snapshot-2016-07-17T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2018-04-15T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2018-08-04T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2016-10-23T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2016-07-02T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2016-08-21T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2017-08-27T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2018-05-13T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2017-09-04T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2016-01-05T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2018-11-11T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2016-03-20T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2018-02-18T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2018-11-18T00_00_00Z.jpg
data/images/CAVITE/fapar_daily/snapshot-2016-10-01T00_00_00Z.jpg
data/images/CAVITE/fapar_

In [8]:
train_x=np.array(x_train_arr)
test_x=np.array(x_test_arr)

train_x=train_x/255.0
test_x=test_x/255.0

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
# Target categorical Variable

y_train = keras.utils.to_categorical(y_train, 2) 
y_test = keras.utils.to_categorical(y_test, 2)

In [14]:
img_rows, img_cols = 224, 224 
input_shape = (img_rows, img_cols, 3)

model = Sequential() 
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape)) 
model.add(Conv2D(64, (3, 3), activation = 'relu')) 
model.add(MaxPooling2D(pool_size = (2, 2))) 
model.add(Dropout(0.25))
model.add(Flatten()) 
model.add(Dense(128, activation = 'relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_3 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 110, 110, 64)     0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 110, 110, 64)      0         
                                                                 
 flatten_1 (Flatten)         (None, 774400)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               99123328  
                                                      

In [15]:
# Compile
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [17]:
# Fit model
model.fit(train_x, y_train, epochs = 100, steps_per_epoch=8, verbose = 2, validation_data = (test_x, y_test), validation_steps=8)

Epoch 1/100
8/8 - 5s - loss: 0.5961 - accuracy: 0.7404 - 5s/epoch - 682ms/step
Epoch 2/100
8/8 - 5s - loss: 0.5939 - accuracy: 0.7019 - 5s/epoch - 677ms/step
Epoch 3/100
8/8 - 5s - loss: 0.5798 - accuracy: 0.7596 - 5s/epoch - 658ms/step
Epoch 4/100
8/8 - 5s - loss: 0.5757 - accuracy: 0.7212 - 5s/epoch - 672ms/step
Epoch 5/100
8/8 - 6s - loss: 0.5087 - accuracy: 0.7981 - 6s/epoch - 691ms/step
Epoch 6/100
8/8 - 6s - loss: 0.4516 - accuracy: 0.7981 - 6s/epoch - 703ms/step
Epoch 7/100
8/8 - 5s - loss: 0.4147 - accuracy: 0.8654 - 5s/epoch - 685ms/step
Epoch 8/100
8/8 - 5s - loss: 0.3975 - accuracy: 0.8750 - 5s/epoch - 686ms/step
Epoch 9/100
8/8 - 5s - loss: 0.3671 - accuracy: 0.8654 - 5s/epoch - 681ms/step
Epoch 10/100
8/8 - 5s - loss: 0.4030 - accuracy: 0.8846 - 5s/epoch - 675ms/step
Epoch 11/100
8/8 - 6s - loss: 0.3770 - accuracy: 0.8269 - 6s/epoch - 700ms/step
Epoch 12/100
8/8 - 5s - loss: 0.3056 - accuracy: 0.9038 - 5s/epoch - 658ms/step
Epoch 13/100
8/8 - 5s - loss: 0.2786 - accuracy: 

In [18]:
# Evaluate Model
score = model.evaluate(test_x, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 3.568575859069824
Test accuracy: 0.6346153616905212


In [20]:
# Predictions
pred = model.predict(test_x) 
pred = np.argmax(pred, axis = 1)[:50] 
label = np.argmax(y_test,axis = 1)[:50] 

print(pred) 
print(label)

2/2 [==============================] - 0s 184ms/step
[0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 1 0 1 1 1 0 0]
[1 0 1 1 0 1 1 1 1 1 0 1 0 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 1 0 1 1
 0 0 1 1 1 1 0 1 0 1 1 0 0]


In [23]:
model.save('models/cvision_model_cavite_fapar_daily.h5')